In [252]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [ ]:
f_in = open(outputModelFile, 'rb')
pickle.dump((dv, model), f_out)
f_out.close()

In [256]:
loaded_model = xgb.Booster()
loaded_model.load_model('model_fraud.model')

In [257]:
model

In [285]:
transaction = {
    'amount': [181.000000],
    'nameOrig': ['C840083671'],
    'oldbalanceOrg': [181.0],
    'nameDest': ['C38997010'],
    'newbalanceOrig': [0.0],
    'oldbalanceDest': [0.0],
    'newbalanceDest': [0.0],
    'step': [1],
    'type': ['TRANSFER']
}


In [286]:
def normalize_input(x):
#     columns = ['step','type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
    numeric_columns = ['step','type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
    X = pd.DataFrame().from_dict(transaction)
    X['isC2C'] = np.array(transaction['nameOrig'][0].startswith('C') & transaction['nameDest'][0].startswith('C')).astype(int)
    X.drop(['nameOrig', 'nameDest'],axis=1, inplace=True)
    return X

In [287]:
X = normalize_input(transaction)
X.info()
X

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   amount          1 non-null      float64
 1   oldbalanceOrg   1 non-null      float64
 2   newbalanceOrig  1 non-null      float64
 3   oldbalanceDest  1 non-null      float64
 4   newbalanceDest  1 non-null      float64
 5   step            1 non-null      int64  
 6   type            1 non-null      object 
 7   isC2C           1 non-null      int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 192.0+ bytes


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,step,type,isC2C
0,181.0,181.0,0.0,0.0,0.0,1,TRANSFER,1


In [288]:
label_encoder = LabelEncoder()
X['type'] = label_encoder.fit_transform(X['type'])

for column in numeric_columns:
    X[column] = np.log1p(X[column])
desired_order = ['amount', 'isC2C', 'newbalanceDest', 'newbalanceOrig', 'oldbalanceDest', 'oldbalanceOrg', 'step', 'type']
X = X.reindex(columns=desired_order)

In [289]:
X

,amount,isC2C,newbalanceDest,newbalanceOrig,oldbalanceDest,oldbalanceOrg,step,type
0,5.204007,1,0.0,0.0,0.0,5.204007,0.693147,0


In [293]:
test_data = xgb.DMatrix(X)
predictionIsFraud = (model.predict(test_data)[0] > 0.5) == 1
predictionIsFraud

True